# 1. 음주 여부에 따라 건강검진 수치 차이가 있을까?

# 2. 신장과 허리둘레의 크기는 체중과 상관관계가 있을까?
- 분석을 통해 가설 검정

# 3. 라이브러리 로드
- 분석에 사용할 pandas
- 수치계산에 사용할 numpy
- 시각화에 사용할 seaborn


#### 구 버전의 주피터 노트북에서는 %matplotlib inline 설정 해야함

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 4. 한글 폰트 설정

In [2]:
# 그래프에서 글자 깨짐 방지 위한 한글 폰트 설정
plt.rc('font', family='Malgun Gothic')

# 마이너스 폰트 깨지는 문제에 대한 대처
plt.rc('axes', unicode_minus=False)

# 레티나 설정을통해 글씨를 더 선명하게 만들고, 폰트 주변 흐릿한 효과 방지
%config InlineBackend.figure_format = 'retina'

# 5. 데이터 불러오기
- 국민 건강 보험 공단_ 건강 검진정보 csv파일
- url : https://www.data.go.kr/data/15007122/fileData.do
- 소개: 건강검진정보란 국민건강보험의 직장가입자와 40세 이상의 피부양자, 세대주인 지역가입자와 40세 이상의 지역가입자의 일반건강검진 결과와 이들 일반건강검진 대상자 중에 만40세와 만66세에 도달한 이들이 받게 되는 생애전환기건강진단 수검이력이 있는 각 연도별 수진자 100만 명에 대한 기본정보(성, 연령대, 시도코드 등)와 검진내역(신장, 체중, 총콜레스테롤, 혈색소 등)으로 구성된 개방데이터입니다.

In [3]:
# 다운로드 받은 파일을 판다스의 read_csv를 통해 읽어오기, encoding = cp949 (한글)
# 파일을 읽어온 후 shape로 행과 열의 수를 출력
data = pd.read_csv('data/국민건강보험공단_건강검진정보_20191231.csv', encoding='cp949')
data.shape

(1000000, 34)

# 5.1 데이터 미리보기

In [4]:
# sample, head, tail을 통해 데이터를 미리보기
data.head()
data.tail()
data.sample()

,기준년도,가입자 일련번호,시도코드,성별코드,연령대 코드(5세단위),신장(5Cm단위),체중(5Kg 단위),허리둘레,시력(좌),시력(우),...,감마 지티피,흡연상태,음주여부,구강검진 수검여부,치아우식증유무,결손치 유무,치아마모증유무,제3대구치(사랑니) 이상,치석,데이터 공개일자
428743,2019,428744,11,2,11,160,70,83.0,1.2,0.9,...,53.0,1.0,NaN,0,NaN,미시행,미시행,미시행,NaN,20191231


# 6. 기본정보보기

In [5]:
# info를 통해 데이터의 크기, 형식, 메모리 사용량 등 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 34 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   기준년도           1000000 non-null  int64  
 1   가입자 일련번호       1000000 non-null  int64  
 2   시도코드           1000000 non-null  int64  
 3   성별코드           1000000 non-null  int64  
 4   연령대 코드(5세단위)   1000000 non-null  int64  
 5   신장(5Cm단위)      1000000 non-null  int64  
 6   체중(5Kg 단위)     1000000 non-null  int64  
 7   허리둘레           999597 non-null   float64
 8   시력(좌)          999805 non-null   float64
 9   시력(우)          999812 non-null   float64
 10  청력(좌)          999819 non-null   float64
 11  청력(우)          999822 non-null   float64
 12  수축기 혈압         994576 non-null   float64
 13  이완기 혈압         994575 non-null   float64
 14  식전혈당(공복혈당)     994477 non-null   float64
 15  총 콜레스테롤        333549 non-null   float64
 16  트리글리세라이드       333544 non-null   float64
 17  HDL 콜레스테롤

In [6]:
# 데이터가 너무 많기 때문에 column 불러오기
data.columns

Index(['기준년도', '가입자 일련번호', '시도코드', '성별코드', '연령대 코드(5세단위)', '신장(5Cm단위)',
       '체중(5Kg 단위)', '허리둘레', '시력(좌)', '시력(우)', '청력(좌)', '청력(우)', '수축기 혈압',
       '이완기 혈압', '식전혈당(공복혈당)', '총 콜레스테롤', '트리글리세라이드', 'HDL 콜레스테롤', 'LDL 콜레스테롤',
       '혈색소', '요단백', '혈청크레아티닌', '(혈청지오티)AST', '(혈청지오티)ALT', '감마 지티피', '흡연상태',
       '음주여부', '구강검진 수검여부', '치아우식증유무', '결손치 유무', '치아마모증유무', '제3대구치(사랑니) 이상',
       '치석', '데이터 공개일자'],
      dtype='object')

In [7]:
# dtypes를 통해 데이터 형식만 출력
data.dtypes

기준년도               int64
가입자 일련번호           int64
시도코드               int64
성별코드               int64
연령대 코드(5세단위)       int64
신장(5Cm단위)          int64
체중(5Kg 단위)         int64
허리둘레             float64
시력(좌)            float64
시력(우)            float64
청력(좌)            float64
청력(우)            float64
수축기 혈압           float64
이완기 혈압           float64
식전혈당(공복혈당)       float64
총 콜레스테롤          float64
트리글리세라이드         float64
HDL 콜레스테롤        float64
LDL 콜레스테롤        float64
혈색소              float64
요단백              float64
혈청크레아티닌          float64
(혈청지오티)AST       float64
(혈청지오티)ALT       float64
감마 지티피           float64
흡연상태             float64
음주여부             float64
구강검진 수검여부          int64
치아우식증유무          float64
결손치 유무            object
치아마모증유무           object
제3대구치(사랑니) 이상     object
치석               float64
데이터 공개일자           int64
dtype: object

#### Part1. 음주 여부에 따라 건강검진 수치 차이가 있을까?
- 라이브러리 로드 및 한글 폰트 설정
- 데이터셋 소개와 로드
----

